# 03. Patrón Observer y Event-Driven
Comunicación asíncrona sin acceso al DOM del navegador.

In [ ]:
// Patrón Observer puro (Event Emitter)

class EventEmitter {
    #eventos = new Map();
    
    on(evento, callback) {
        if (!this.#eventos.has(evento)) {
            this.#eventos.set(evento, []);
        }
        this.#eventos.get(evento).push(callback);
        return () => this.off(evento, callback); // Retorna función de desuscripción
    }
    
    emit(evento, datos) {
        const listeners = this.#eventos.get(evento) || [];
        listeners.forEach(cb => {
            try {
                cb(datos);
            } catch (e) {
                console.error(`Error en listener de ${evento}:`, e);
            }
        });
    }
    
    off(evento, callback) {
        if (!this.#eventos.has(evento)) return;
        const listeners = this.#eventos.get(evento).filter(cb => cb !== callback);
        this.#eventos.set(evento, listeners);
    }
    
    once(evento, callback) {
        const wrapper = (datos) => {
            this.off(evento, wrapper);
            callback(datos);
        };
        this.on(evento, wrapper);
    }
}

// Aplicación: Sistema de logging de operaciones matemáticas
class CalculadoraAuditable extends EventEmitter {
    #operaciones = [];
    
    ejecutar(operacion, a, b) {
        let resultado;
        const timestamp = Date.now();
        
        switch(operacion) {
            case 'suma': resultado = a + b; break;
            case 'resta': resultado = a - b; break;
            case 'multiplicacion': resultado = a * b; break;
            case 'division': 
                if (b === 0) {
                    this.emit('error', {tipo: 'DivisionZero', a, b});
                    throw new Error('División por cero');
                }
                resultado = a / b;
                break;
            default:
                this.emit('error', {tipo: 'OperacionInvalida', operacion});
                throw new Error('Operación no soportada');
        }
        
        const log = {operacion, a, b, resultado, timestamp};
        this.#operaciones.push(log);
        
        // Emitir eventos
        this.emit('operacion', log);
        if (resultado < 0) this.emit('resultado_negativo', log);
        
        return resultado;
    }
    
    getHistorial() {
        return [...this.#operaciones];
    }
}

// Uso y suscripción
const calc = new CalculadoraAuditable();
const logs = [];
const errores = [];

// Suscriptores
calc.on('operacion', (data) => {
    logs.push(`? ${data.operacion}: ${data.a} op ${data.b} = ${data.resultado}`);
});

calc.on('resultado_negativo', (data) => {
    logs.push(`?? Alerta: Resultado negativo detectado (${data.resultado})`);
});

calc.on('error', (err) => {
    errores.push(`Error ${err.tipo}: ${JSON.stringify(err)}`);
});

// Ejecutar operaciones
calc.ejecutar('suma', 5, 3);
calc.ejecutar('resta', 10, 15); // Disparará resultado_negativo
calc.ejecutar('multiplicacion', 4, 5);

try {
    calc.ejecutar('division', 10, 0);
} catch(e) {
    // Error capturado, pero el evento ya se emitió
}

// Despliegue de resultados
({
    "application/json": {
        eventos_capturados: logs,
        errores_capturados: errores,
        historial_completo: calc.getHistorial(),
        patron: "Observer: Desacoplamiento entre emisor y receptores"
    }
});

In [ ]:
// Async/Await y Promesas en entorno educativo
// Simulando llamadas a API o cálculos pesados

function operacionAsincrona(ms, exito = true) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            if (exito) {
                resolve({status: 'ok', tiempo: ms, id: Math.random().toString(36).substr(2, 9)});
            } else {
                reject(new Error('Fallo simulado de red'));
            }
        }, ms);
    });
}

// Secuencial vs Paralelo
async function demoAsync() {
    const resultados = [];
    
    // Secuencial (lento)
    const inicio1 = Date.now();
    const r1 = await operacionAsincrona(100, true);
    const r2 = await operacionAsincrona(100, true);
    const tiempoSecuencial = Date.now() - inicio1;
    
    // Paralelo (rápido)
    const inicio2 = Date.now();
    const [r3, r4] = await Promise.all([
        operacionAsincrona(100, true),
        operacionAsincrona(100, true)
    ]);
    const tiempoParalelo = Date.now() - inicio2;
    
    // Manejo de errores con Promise.allSettled
    const mixto = await Promise.allSettled([
        operacionAsincrona(50, true),
        operacionAsincrona(50, false), // Fallará
        operacionAsincrona(50, true)
    ]);
    
    return {
        secuencial: {tiempo: tiempoSecuencial, resultados: [r1, r2]},
        paralelo: {tiempo: tiempoParalelo, resultados: [r3, r4]},
        tolerancia_fallas: mixto.map(m => ({
            status: m.status,
            valor: m.status === 'fulfilled' ? m.value : m.reason.message
        }))
    };
}

// Ejecutar (el kernel JS de JupyterLite soporta top-level await)
demoAsync();